In [18]:
import sys
import HTMLParser
no_labels = "./test.nolabels.txt"
with_labels = "./test_withlabels.txt"

In [19]:
with open(no_labels) as t:
    input_lines = t.readlines()
    tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                sentence = ' '.join(str(x) for x in arr) 
                tweets.append(sentence.strip())
                arr = []
            else:
                word = line.strip()
                arr.append(word)
        except ValueError:
            continue

with open(with_labels) as t:
    input_lines = t.readlines()
    tweets_new = []
    arr = []
    arr2 = []
    tags = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                sentence = ' '.join(str(x) for x in arr) 
                tweets_new.append(sentence.strip())
                tags.append(arr2)
                arr = []
                arr2 = []
            else:
                (word, tag) = line.split()
                arr.append(word)
                arr2.append(tag)
        except ValueError:
            continue

print len(tweets)
print len(tweets_new)
# with open(extra) as e:
#     input_lines = e.readlines()
#     sentences = []
#     arr_extra = []
#     pairs = []
#     arr_extra_pairs = []
#     for line in input_lines:
#         try:
#             if line in ['\n', '\r\n']:
#                 sentence = ' '.join(str(x) for x in arr_extra)
#                 sentences.append(sentence.strip())
#                 pairs.append(arr_extra_pairs)
#                 arr_extra = []
#                 arr_extra_pairs = []
#             else:
#                 (word, tag) = line.split()
#                 if('B' in tag):
#                     tag = 'B'
#                 elif('I' in tag):
#                     tag = 'I'
#                 arr_extra.append(word)
#                 arr_extra_pairs.append([word, tag])
#         except ValueError:
#             continue

count = 0
new_test = []
new_test_sentences = []
i = 0
for t in tweets:
    if(t not in tweets_new):
        count += 1
        new_test_sentences.append(t)
        #new_test.append(pairs[i])
    i += 1


print '# not in withlabels file: ' + str(count)

open("./test_small_withlabels.txt", 'w').close()
with open ('./test_small_withlabels.txt', 'w') as f: 
    for t in tweets:
        if(t in tweets_new):
            index = tweets_new.index(t)
            tweet_tags = tags[index]
            
            tweet = t.split()
            for i in range(len(tweet)):
                f.write(tweet[i] + "\t" + tweet_tags[i] + "\n")
            f.write("\n")
                
        

# open(tweebo_file, 'w').close()
# with open (tweebo_file, 'w') as f: 
#     for s in new_test_sentences:
#         f.write(s)
#         f.write("\n") 
            
# open(extra_labeled, 'w').close()
# with open (extra_labeled, 'w') as f: 
#     for i in new_test:
#         for j in i:
#             f.write(j[0] + " " + j[1] + "\n")
#         f.write("\n") 


2377
3336
# not in withlabels file: 0


In [14]:
print tweets[0]
print tweets_new[0]

Kill or be killed . This is life .
Kill or be killed . This is life .


In [45]:
# Setup Train/Test Tweebo files to extract features

file_to_train = extra_labeled
file_to_train_tweebo = tweebo_features
result_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/Python/extra_tweebo_features.txt"
kaggle = False
with open(file_to_train) as train:
    input_lines = train.readlines()
    labeled_tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                labeled_tweets.append(arr)
                arr = []
            else:
                if not kaggle:
                    (word, tag) = line.split()
                    arr.append((word, tag))
                else:
                    arr.append([line.strip()])
                
        except ValueError:
            continue

with open(file_to_train_tweebo) as train_tweebo:
    input_lines = train_tweebo.readlines()
    tweebo_tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                tweebo_tweets.append(arr)
                arr = []
            else:
                (word_id, word, lemma, cpostag, postag, feats, head, dep_relation) = line.split()
                arr.append((word, word_id, postag, head, dep_relation));
        except ValueError:
            continue
            
features = []

for s1, s2 in zip(labeled_tweets, tweebo_tweets):
    tweebo_index = 0
    sentence = []
    head_diff = 0
    head_point = 0
    head_point_arr = []
    head_diff_arr = []
    for i in range(len(s1)):
        word1 = s1[i][0]
        escaped_word1 = HTMLParser.HTMLParser().unescape(s1[i][0]) 
        word2 = HTMLParser.HTMLParser().unescape(s2[tweebo_index][0]) 
        t_ind = tweebo_index
        if(word1 != word2 and escaped_word1 != word2):
            head_point = i+1
            head_point_arr.append(head_point)
            while True:
                if(word2 == word1 or word2 == escaped_word1):
                    break;
                else:
                    head_diff += 1
                    if(int(s2[tweebo_index][3]) == -1 and int(s2[t_ind][3]) == -1):
                        t_ind += 1
                    tweebo_index += 1
                    word2 += HTMLParser.HTMLParser().unescape(s2[tweebo_index][0])
            head_diff_arr.append(head_diff)
            
        if not kaggle:
            sentence.append([i+1, escaped_word1, s1[i][1], s2[t_ind][2], int(s2[t_ind][3]), s2[t_ind][4]])
        else:
            sentence.append([i+1, escaped_word1, s2[t_ind][2], int(s2[t_ind][3]), s2[t_ind][4]])
        tweebo_index += 1
    #head indexes reconciliation
    for word in sentence:
        if not kaggle:
            old_head = word[4]
        else:
            old_head = word[3]
            
        for h in range(len(head_point_arr)):
            if(old_head < head_point_arr[h]):
                break;
            elif(old_head > head_point_arr[h] and h == (len(head_point_arr) - 1)):
                if not kaggle:
                    word[4] = old_head - head_diff_arr[h]
                else:
                    word[3] = old_head - head_diff_arr[h]
                
            elif(old_head > head_point_arr[h] and old_head < head_point_arr[h+1]):
                if not kaggle:
                    word[4] = old_head - head_diff_arr[h]
                else:
                    word[3] = old_head - head_diff_arr[h]
            else:
                continue
    
    features.append(sentence)
    
open(result_file, 'w').close()
with open (result_file, 'w') as f: 
    for tweet in features:
        for word in tweet:
            f.write(' '.join(str(x) for x in word).strip())
            f.write("\n")
        f.write("\n")

In [48]:
with open('C:\Users\User\Documents\Cornell\Courses\NLP\HW4\data\kaggle_crf_result.txt') as t:
    input_lines = t.readlines()
    arr = []
    for line in input_lines:
        arr.append(line)
            
with open('C:\Users\User\Documents\Cornell\Courses\NLP\HW4\data\kaggle_crf_result2.txt') as t2:
    input_lines = t2.readlines()
    arr2 = []
    for line in input_lines:
        arr2.append(line)

count = 0
for i in range(len(arr2)):
    if(arr2[i] != arr[i]):
        count += 1
        
print count

0
